<a href="https://colab.research.google.com/github/sjkchang/CMPE255-Assignment-2/blob/master/notebooks/Assignment2_AnomalyDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret
!pip install pycaret[analysis]


In [6]:
import pandas as pd

In [2]:
# check installed version
import pycaret
pycaret.__version__

'3.0.4'

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Datasets/255-Datasets
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Datasets/255-Datasets
 3D-Print-Anomoly-Detection/
 Amazon-stock-time-series.csv
 CarPrice_Assignment.csv
'Credit Card Customer Data - Clustering.csv'
 falling-elders-anomoly-detection/
 flagged/
 Groceries_dataset-association-rules-mining.csv
 heartattack.csv
 logs.log
 Mine_Dataset.csv
 my_first_pipeline.pkl
 ny-traffic.csv
'Out-of-Sample Forecast Plot.html'
 secondhand-car-price-regression.csv


In [4]:
# loading sample dataset from pycaret dataset module
from pycaret.datasets import get_data
data = get_data('ny-traffic')

,Unnamed: 0,timestamp,value
0,0,2014-07-01 00:00:00,10844
1,1,2014-07-01 00:30:00,8127
2,2,2014-07-01 01:00:00,6210
3,3,2014-07-01 01:30:00,4656
4,4,2014-07-01 02:00:00,3820


In [11]:
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [8]:
data

,Unnamed: 0,timestamp,value
0,0,2014-07-01 00:00:00,10844
1,1,2014-07-01 00:30:00,8127
2,2,2014-07-01 01:00:00,6210
3,3,2014-07-01 01:30:00,4656
4,4,2014-07-01 02:00:00,3820
...,...,...,...
10315,10315,2015-01-31 21:30:00,24670
10316,10316,2015-01-31 22:00:00,25721
10317,10317,2015-01-31 22:30:00,27309
10318,10318,2015-01-31 23:00:00,26591


In [18]:
# import pycaret anomaly and init setup
from pycaret.anomaly import *
s = setup(data, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(10320, 3)"
2,Transformed data shape,"(10320, 5)"
3,Numeric features,2
4,Date features,1
5,Preprocess,True
6,Imputation type,simple
7,Numeric imputation,mean
8,Categorical imputation,mode
9,CPU Jobs,-1


In [54]:
# train iforest model
model = create_model('iforest', fraction=0.01)
model

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

IForest(behaviour='new', bootstrap=False, contamination=0.01,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=123, verbose=0)

In [55]:
# predict on test set
prediction = predict_model(model, data=data)
prediction

,Unnamed: 0,timestamp_year,timestamp_month,timestamp_day,value,Anomaly,Anomaly_Score
0,0.0,2014,7,1,10844.0,0,-0.061478
1,1.0,2014,7,1,8127.0,0,-0.052599
2,2.0,2014,7,1,6210.0,0,-0.046234
3,3.0,2014,7,1,4656.0,0,-0.039880
4,4.0,2014,7,1,3820.0,0,-0.031504
...,...,...,...,...,...,...,...
10315,10315.0,2015,1,31,24670.0,1,0.029365
10316,10316.0,2015,1,31,25721.0,1,0.035147
10317,10317.0,2015,1,31,27309.0,1,0.050100
10318,10318.0,2015,1,31,26591.0,1,0.044981


In [56]:
datetime_series = pd.to_datetime(prediction[['timestamp_year', 'timestamp_month', 'timestamp_day']].rename(columns={'timestamp_year': 'year', 'timestamp_month': 'month', 'timestamp_day': 'day'}))
prediction['timestamp'] = datetime_series
prediction

,Unnamed: 0,timestamp_year,timestamp_month,timestamp_day,value,Anomaly,Anomaly_Score,timestamp
0,0.0,2014,7,1,10844.0,0,-0.061478,2014-07-01
1,1.0,2014,7,1,8127.0,0,-0.052599,2014-07-01
2,2.0,2014,7,1,6210.0,0,-0.046234,2014-07-01
3,3.0,2014,7,1,4656.0,0,-0.039880,2014-07-01
4,4.0,2014,7,1,3820.0,0,-0.031504,2014-07-01
...,...,...,...,...,...,...,...,...
10315,10315.0,2015,1,31,24670.0,1,0.029365,2015-01-31
10316,10316.0,2015,1,31,25721.0,1,0.035147,2015-01-31
10317,10317.0,2015,1,31,27309.0,1,0.050100,2015-01-31
10318,10318.0,2015,1,31,26591.0,1,0.044981,2015-01-31


In [57]:
anomolies = prediction.loc[prediction['Anomaly'] == 1]
anomoly_dates = set(anomolies['timestamp'])
anomoly_dates

{Timestamp('2015-01-01 00:00:00'),
 Timestamp('2015-01-02 00:00:00'),
 Timestamp('2015-01-03 00:00:00'),
 Timestamp('2015-01-23 00:00:00'),
 Timestamp('2015-01-29 00:00:00'),
 Timestamp('2015-01-30 00:00:00'),
 Timestamp('2015-01-31 00:00:00')}